In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Load data
df = pd.read_csv("modified.csv")

# Define target
target = 'Life expectancy'

# Define column roles
categorical_cols = ['Gender']
ignore_cols = ['Country', 'Year', 'Year.1', target]
numerical_cols = [col for col in df.columns if col not in categorical_cols + ignore_cols]

# Encode categorical variables
df['Gender'] = LabelEncoder().fit_transform(df['Gender'])

# Scale numerical features
scaler = StandardScaler()
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

# Final feature set
X = df[categorical_cols + numerical_cols]
y = df[target]

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [2]:
import torch
import rtdl
from rtdl import TabTransformerRegressor
from torch import nn
from torch.utils.data import TensorDataset, DataLoader


# Prepare input tensors
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).unsqueeze(1).to(device)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).unsqueeze(1).to(device)

# Get unique values for categorical columns (needed for embedding sizes)
n_categories = [df[col].nunique() for col in categorical_cols]

# Define the model
model = TabTransformerRegressor(
    n_continuous=len(numerical_cols),
    n_classes=1,
    n_categories=n_categories,
    d_token=64,
    n_layers=3,
    n_heads=4,
    dropout=0.1
)


ImportError: cannot import name 'TabTransformerRegressor' from 'rtdl' (/usr/local/lib/python3.11/dist-packages/rtdl/__init__.py)

In [ ]:
# Optimizer and loss
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)
loss_fn = nn.MSELoss()

# Dataloader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)

# Training loop
epochs = 20
model.train()

for epoch in range(epochs):
    total_loss = 0
    for xb, yb in train_loader:
        optimizer.zero_grad()
        preds = model(xb)
        loss = loss_fn(preds, yb)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader):.4f}")


In [4]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# MODEL
model = TabTransformer(
    categories=(2,),  # 'Gender' column has 2 unique values
    num_continuous=X_train.shape[1] - 1,  # excluding the categorical
    dim=64,
    dim_out=1,
    depth=4,
    heads=4,
    attn_dropout=0.1,
    ff_dropout=0.1
).to(device)

# INPUT TENSORS
X_categ_train = torch.tensor(X_train.iloc[:, 0].values, dtype=torch.long).unsqueeze(1).to(device)
X_cont_train = torch.tensor(X_train.iloc[:, 1:].values, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).unsqueeze(1).to(device)

X_categ_test = torch.tensor(X_test.iloc[:, 0].values, dtype=torch.long).unsqueeze(1).to(device)
X_cont_test = torch.tensor(X_test.iloc[:, 1:].values, dtype=torch.float32).to(device)

# DATALOADER
train_dataset = TensorDataset(X_categ_train, X_cont_train, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)

# TRAINING
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

model.train()
for epoch in range(10):
    total_loss = 0
    for x_cat, x_cont, yb in train_loader:
        optimizer.zero_grad()
        preds = model(x_cat, x_cont)
        loss = loss_fn(preds, yb)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}: Loss = {total_loss:.4f}")


Epoch 1: Loss = 22.4620
Epoch 2: Loss = 8.6705
Epoch 3: Loss = 7.1800
Epoch 4: Loss = 6.3039
Epoch 5: Loss = 5.8419
Epoch 6: Loss = 5.2850
Epoch 7: Loss = 4.8395
Epoch 8: Loss = 4.4000
Epoch 9: Loss = 4.0517
Epoch 10: Loss = 3.5211


In [8]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

model.eval()
with torch.no_grad():
    preds = model(X_categ_test, X_cont_test).cpu().numpy().flatten()
    actual = y_test_tensor.cpu().numpy().flatten()

mae = mean_absolute_error(actual, preds)
mse = mean_squared_error(actual, preds)
rmse = np.sqrt(mse)
r2 = r2_score(actual, preds)

print(f"\nMAE:  {mae:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"R²:   {r2:.4f}")



MAE:  0.1837
RMSE: 0.2519
R²:   0.9352
